In [1]:
import pandas as pd
import re
import unicodedata

def enlever_accents(text):
    """Enlève les accents d'un texte."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(text))
        if unicodedata.category(c) != 'Mn'
    )

def nettoyer_texte(text):
    """Met le texte en minuscules et enlève les accents."""
    return enlever_accents(text.lower())

def nettoyer_colonnes(cols):
    """Nettoie une liste de noms de colonnes : minuscules, sans accents, espaces remplacés par '_'."""
    return [nettoyer_texte(c).replace(' ', '_') for c in cols]

def detecter_sexe(titre_str):
    """Détecte le sexe dans le titre, retourne 'Masculin', 'Feminin', 'Ensemble', ou '.' si non trouvé."""
    t = nettoyer_texte(titre_str)
    if 'masculin' in t:
        return 'Masculin'
    elif 'feminin' in t:
        return 'Feminin'
    elif 'ensemble sexes' in t or 'ensemble sexe' in t:
        return 'Ensemble'
    return '.'  # Pas d'info

def detecter_milieu(titre_str):
    """Détecte le milieu (communal, non communal, ensemble), retourne 'C', 'NC', 'Ensemble', ou '.' si non trouvé."""
    t = nettoyer_texte(titre_str)
    if 'non communal' in t:
        return 'NC'
    elif 'communal' in t:
        return 'C'
    elif 'ensemble milieux' in t or 'ensemble milieu' in t:
        return 'Ensemble'
    return '.'  # Pas d'info

def detecter_unite(titre_str):
    """Détecte l'unité des données (pourcentage ou effectif). Par défaut 'effectif'."""
    t = nettoyer_texte(titre_str)
    if '%' in titre_str or 'pourcentage' in t:
        return '%'
    elif 'nombre' in t or 'effectif' in t:
        return 'effectif'
    return 'effectif'  # Valeur par défaut

def trouver_colonne_similaire(df, mots_cles):
    """Recherche colonne contenant tous les mots clés dans son nom."""
    for col in df.columns:
        col_nettoyee = nettoyer_texte(col)
        if all(mot in col_nettoyee for mot in mots_cles):
            return col
    return None

# Charger le fichier Excel
fichier_excel = 'educ.xlsx'
xls = pd.ExcelFile(fichier_excel)

# Feuilles T12-90 à T72-122 (exemple selon ta demande)
pattern = re.compile(r'^T\d{2}-(\d{2,3})$')
feuilles = [f for f in xls.sheet_names if pattern.match(f) and 94 <= int(pattern.match(f).group(1)) <= 122]

dfs_long = []

for feuille in feuilles:
    try:
        print(f"📄 Traitement de la feuille : {feuille}")

        # Lire les 2 premières lignes comme titre
        titres = pd.read_excel(xls, sheet_name=feuille, nrows=2, header=None)
        titre_str = ' '.join(titres.astype(str).fillna('').values.flatten())

        # Détection métadonnées
        sexe = detecter_sexe(titre_str)
        milieu = detecter_milieu(titre_str)
        unite = detecter_unite(titre_str)

        # Année (cherche un nombre 19xx ou 20xx dans le titre)
        annee_match = re.search(r'\b(19\d{2}|20\d{2})\b', titre_str)
        if annee_match:
            annee = int(annee_match.group(1))
        else:
            match = pattern.match(feuille)
            annee = int("19" + match.group(1)) if match else '.'

        # Lire les données après les 2 lignes de titre
        df = pd.read_excel(xls, sheet_name=feuille, skiprows=2)
        df.columns = nettoyer_colonnes(df.columns)

        # Trouver colonne gouvernorat
        gouv_col = trouver_colonne_similaire(df, ['gouver'])
        if gouv_col is None:
            df['gouvernorat'] = '.'
            gouv_col = 'gouvernorat'
        else:
            df[gouv_col] = df[gouv_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        # Trouver colonne groupe d'âge (âge)
        groupe_age_col = trouver_colonne_similaire(df, ['groupe', 'age'])
        if groupe_age_col is None:
            # Essayons juste 'age' seul
            groupe_age_col = trouver_colonne_similaire(df, ['age'])
        if groupe_age_col is None:
            df['groupe_d_age'] = '.'
            groupe_age_col = 'groupe_d_age'
        else:
            df[groupe_age_col] = df[groupe_age_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        # Colonnes niveaux d'étude = toutes sauf gouvernorat, groupe d'âge, total
        colonnes_instruction = [c for c in df.columns if c not in [gouv_col, groupe_age_col, 'total']]

        # Transformer en format long
        df_long = df.melt(
            id_vars=[gouv_col, groupe_age_col],
            value_vars=colonnes_instruction,
            var_name='niveaux_d_etude',
            value_name='valeur'
        )

        # Uniformiser noms colonnes gouvernorat et groupe d'âge
        if gouv_col != 'gouvernorat':
            df_long = df_long.rename(columns={gouv_col: 'gouvernorat'})
        if groupe_age_col != 'groupe_d_age':
            df_long = df_long.rename(columns={groupe_age_col: 'groupe_d_age'})

        # Ajout colonnes fixes et métadonnées
        df_long.insert(0, 'indicateur', 'Population ')
        df_long['sexe'] = sexe
        df_long['milieu'] = milieu
        df_long['annee'] = annee
        df_long['unite'] = unite
        df_long['feuille'] = feuille

        dfs_long.append(df_long)
        print(f"✅ Feuille '{feuille}' traitée.\n")

    except Exception as e:
        print(f"❌ Erreur dans la feuille '{feuille}': {e}\n")

if dfs_long:
    df_final = pd.concat(dfs_long, ignore_index=True)
    df_final = df_final[~df_final['groupe_d_age'].str.lower().str.contains(r'\bage\b')]
    df_final = df_final.drop_duplicates()
    print("✅ Fusion terminée. Exemple :\n")
    print(df_final.head())
    df_final.to_excel("resultat_educ_auto.xlsx", index=False)
    print("📁 Données enregistrées dans 'resultat_educ_auto.xlsx'")

    # Optionnel : affichage complet
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(df_final)

    df_final.to_csv("resultat_4.csv", index=False, encoding='utf-8-sig')
else:
    print("❌ Aucune donnée traitée.")


📄 Traitement de la feuille : T72-94
✅ Feuille 'T72-94' traitée.

📄 Traitement de la feuille : T72-95
✅ Feuille 'T72-95' traitée.

📄 Traitement de la feuille : T72-96
✅ Feuille 'T72-96' traitée.

📄 Traitement de la feuille : T72-97
✅ Feuille 'T72-97' traitée.

📄 Traitement de la feuille : T72-98
✅ Feuille 'T72-98' traitée.

📄 Traitement de la feuille : T72-99
✅ Feuille 'T72-99' traitée.

📄 Traitement de la feuille : T72-100
✅ Feuille 'T72-100' traitée.

📄 Traitement de la feuille : T72-101
✅ Feuille 'T72-101' traitée.

📄 Traitement de la feuille : T72-102
✅ Feuille 'T72-102' traitée.

📄 Traitement de la feuille : T72-103
✅ Feuille 'T72-103' traitée.

📄 Traitement de la feuille : T72-104
✅ Feuille 'T72-104' traitée.

📄 Traitement de la feuille : T72-105
✅ Feuille 'T72-105' traitée.

📄 Traitement de la feuille : T72-106
✅ Feuille 'T72-106' traitée.

📄 Traitement de la feuille : T72-107
✅ Feuille 'T72-107' traitée.

📄 Traitement de la feuille : T72-108
✅ Feuille 'T72-108' traitée.

📄 Trait

# exclut feuille T72-90 et T72-92

In [2]:
import pandas as pd
import re
import unicodedata

def enlever_accents(text):
    """Enlève les accents d'un texte."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(text))
        if unicodedata.category(c) != 'Mn'
    )

def nettoyer_texte(text):
    """Met le texte en minuscules et enlève les accents."""
    return enlever_accents(text.lower())

def nettoyer_colonnes(cols):
    """Nettoie une liste de noms de colonnes : minuscules, sans accents, espaces remplacés par '_'."""
    return [nettoyer_texte(c).replace(' ', '_') for c in cols]

def detecter_sexe(titre_str):
    """Détecte le sexe dans le titre, retourne 'Masculin', 'Feminin', 'Ensemble', ou '.' si non trouvé."""
    t = nettoyer_texte(titre_str)
    if 'masculin' in t:
        return 'Masculin'
    elif 'feminin' in t:
        return 'Feminin'
    elif 'ensemble sexes' in t or 'ensemble sexe' in t:
        return 'Ensemble'
    return '.'  # Pas d'info

def detecter_milieu(titre_str):
    """Détecte le milieu (communal, non communal, ensemble), retourne 'C', 'NC', 'Ensemble', ou '.' si non trouvé."""
    t = nettoyer_texte(titre_str)
    if 'non communal' in t:
        return 'NC'
    elif 'communal' in t:
        return 'C'
    elif 'ensemble milieux' in t or 'ensemble milieu' in t:
        return 'Ensemble'
    return '.'  # Pas d'info

def detecter_unite(titre_str):
    """Détecte l'unité des données (pourcentage ou effectif). Par défaut 'effectif'."""
    t = nettoyer_texte(titre_str)
    if '%' in titre_str or 'pourcentage' in t:
        return '%'
    elif 'nombre' in t or 'effectif' in t:
        return 'effectif'
    return 'effectif'  # Valeur par défaut

def trouver_colonne_similaire(df, mots_cles):
    """Recherche colonne contenant tous les mots clés dans son nom."""
    for col in df.columns:
        col_nettoyee = nettoyer_texte(col)
        if all(mot in col_nettoyee for mot in mots_cles):
            return col
    return None

# Charger le fichier Excel
fichier_excel = 'educ.xlsx'
xls = pd.ExcelFile(fichier_excel)

# Feuilles T12-90 à T72-122
pattern = re.compile(r'^T\d{2}-(\d{2,3})$')
feuilles = [f for f in xls.sheet_names if pattern.match(f) and 94 <= int(pattern.match(f).group(1)) <= 122]

dfs_long = []

for feuille in feuilles:
    try:
        print(f"📄 Traitement de la feuille : {feuille}")

        # Lire les 2 premières lignes comme titre
        titres = pd.read_excel(xls, sheet_name=feuille, nrows=2, header=None)
        titre_str = ' '.join(titres.astype(str).fillna('').values.flatten())

        # Détection métadonnées
        sexe = detecter_sexe(titre_str)
        milieu = detecter_milieu(titre_str)
        unite = detecter_unite(titre_str)

        # Détection indicateur depuis le premier mot
        mots_titre = nettoyer_texte(titre_str).split()
        indicateur = mots_titre[0].capitalize() if mots_titre else 'Population'

        # Année (cherche un nombre 19xx ou 20xx dans le titre)
        annee_match = re.search(r'\b(19\d{2}|20\d{2})\b', titre_str)
        if annee_match:
            annee = int(annee_match.group(1))
        else:
            match = pattern.match(feuille)
            annee = int("19" + match.group(1)) if match else '.'

        # Lire les données après les 2 lignes de titre
        df = pd.read_excel(xls, sheet_name=feuille, skiprows=2)
        df.columns = nettoyer_colonnes(df.columns)

        # Trouver colonne gouvernorat
        gouv_col = trouver_colonne_similaire(df, ['gouver'])
        if gouv_col is None:
            df['gouvernorat'] = '.'
            gouv_col = 'gouvernorat'
        else:
            df[gouv_col] = df[gouv_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        # Trouver colonne groupe d'âge (âge)
        groupe_age_col = trouver_colonne_similaire(df, ['groupe', 'age'])
        if groupe_age_col is None:
            groupe_age_col = trouver_colonne_similaire(df, ['age'])
        if groupe_age_col is None:
            df['groupe_d_age'] = '.'
            groupe_age_col = 'groupe_d_age'
        else:
            df[groupe_age_col] = df[groupe_age_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        # Colonnes niveaux d'étude = toutes sauf gouvernorat, groupe d'âge, total
        colonnes_instruction = [c for c in df.columns if c not in [gouv_col, groupe_age_col, 'total']]

        # Transformer en format long
        df_long = df.melt(
            id_vars=[gouv_col, groupe_age_col],
            value_vars=colonnes_instruction,
            var_name='niveaux_d_etude',
            value_name='valeur'
        )

        # Uniformiser noms colonnes gouvernorat et groupe d'âge
        if gouv_col != 'gouvernorat':
            df_long = df_long.rename(columns={gouv_col: 'gouvernorat'})
        if groupe_age_col != 'groupe_d_age':
            df_long = df_long.rename(columns={groupe_age_col: 'groupe_d_age'})

        # Ajout colonnes fixes et métadonnées
        df_long.insert(0, 'indicateur', indicateur)
        df_long['sexe'] = sexe
        df_long['milieu'] = milieu
        df_long['annee'] = annee
        df_long['unite'] = unite
        df_long['feuille'] = feuille

        dfs_long.append(df_long)
        print(f"✅ Feuille '{feuille}' traitée.\n")

    except Exception as e:
        print(f"❌ Erreur dans la feuille '{feuille}': {e}\n")

if dfs_long:
    df_final = pd.concat(dfs_long, ignore_index=True)
    df_final = df_final[~df_final['groupe_d_age'].str.lower().str.contains(r'\bage\b')]
    # Remplacement de mots dans la colonne indicateurdétudes
    df_final['indicateur'] = df_final['indicateur'].str.replace(r'\b[Rr]epartition\b', ' Répartition proportionnelle (%) de la Population fréquentant un établissement scolaire selon le cycle détudes', regex=True)
    df_final['indicateur'] = df_final['indicateur'].str.replace(r'\b[Pp]opulation\b', 'Population fréquentant un établissement scolaire selon le cycle ', regex=True)

    df_final = df_final.drop_duplicates()
    print("✅ Fusion terminée. Exemple :\n")
    print(df_final.head())

    df_final.to_excel("resultat_educ_auto.xlsx", index=False)
    print("📁 Données enregistrées dans 'resultat_educ_auto.xlsx'")

    df_final.to_csv("resultat_4.csv", index=False, encoding='utf-8-sig')
else:
    print("❌ Aucune donnée traitée.")


📄 Traitement de la feuille : T72-94
✅ Feuille 'T72-94' traitée.

📄 Traitement de la feuille : T72-95
✅ Feuille 'T72-95' traitée.

📄 Traitement de la feuille : T72-96
✅ Feuille 'T72-96' traitée.

📄 Traitement de la feuille : T72-97
✅ Feuille 'T72-97' traitée.

📄 Traitement de la feuille : T72-98
✅ Feuille 'T72-98' traitée.

📄 Traitement de la feuille : T72-99
✅ Feuille 'T72-99' traitée.

📄 Traitement de la feuille : T72-100
✅ Feuille 'T72-100' traitée.

📄 Traitement de la feuille : T72-101
✅ Feuille 'T72-101' traitée.

📄 Traitement de la feuille : T72-102
✅ Feuille 'T72-102' traitée.

📄 Traitement de la feuille : T72-103
✅ Feuille 'T72-103' traitée.

📄 Traitement de la feuille : T72-104
✅ Feuille 'T72-104' traitée.

📄 Traitement de la feuille : T72-105
✅ Feuille 'T72-105' traitée.

📄 Traitement de la feuille : T72-106
✅ Feuille 'T72-106' traitée.

📄 Traitement de la feuille : T72-107
✅ Feuille 'T72-107' traitée.

📄 Traitement de la feuille : T72-108
✅ Feuille 'T72-108' traitée.

📄 Trait